# Exploring Landsat Data and GEE

In [4]:
#imports

import ee
import geemap
from geemap import *
import json
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import os

### Visualisation

In [5]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [6]:
#adding Landsat8 data to the Map
landsat_8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate('2013-09-01', '2014-05-31') #filtered for the wet seasons

#define visualisation parameters
image_vis_params = {
    'bands': ['B4', 'B3', 'B2'], #red, green, blue bands
    'max': 0.25 #reflectance
} 

#set map centre
Map.setCenter(28.75, -0.66667, 8) #centering map on North Kivu Region, DRC

#display imagery
Map.addLayer(landsat_8,image_vis_params , 'Landsat True Colour Composite')
#finding out the data type
#print('landsat_8', landsat_8 )
Map

Map(center=[-0.66667, 28.75], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [13]:
#Adding AOI GeoJSON
file_path = os.path.abspath('/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/GEE_examples/north_kivu.json')

with open(file_path) as f:
    json_data = json.load(f)

north_kivu_DRC = GeoJSON(data=json_data, name='North_Kivu_Region')
Map.add_layer(north_kivu_DRC)


In [ ]:
#retrieving the image in the collection with the least cloud cover
#sorted = landsat_8.sort('CLOUD_COVER')
#scene = sorted.first()
#Map.addLayer(scene, {}, 'default RGB');


### Compositing, Masking, and Mosaicking


#### Compositing with Reducers

- Reducing an image collection to an image is a temporal reduction when the collection represents images over time. 

- Landsat 8 visits the same spot on the Earth every 16 days. That means that over a 6 month period, there will be approximately 12 images (and more where the scenes overlap)

- Merely adding the collection to the map results in selecting the most recent pixel - the one from the latest image in the stack.

- Rather than take the most recent pixel from the stack, Earth Engine can be instructed to pick the median value in the stack. 

- This has the benefit of removing clouds (which have a high value) and shadows (which have a low value).

- When an image collection is reduced using the median reducer, the composite value is the median in each band, over time.

In [8]:
#Get the median over time, in each band, in each pixel
median = landsat_8.median()

#define visualisation parameters
image_vis_params = {
    'bands': ['B4', 'B3', 'B2'], #red, green, blue bands
    'max': 0.25 #reflectance
} 

#display the median composit
Map.addLayer(median, image_vis_params, 'median')

Map.setCenter(28.75, -0.66667, 8) #centering map on North Kivu Region, DRC

Map

Map(bottom=33189.0, center=[-0.66667, 28.75], controls=(WidgetControl(options=['position', 'transparent_bg'], …

#### Creating a simple cloud-free composite


In [9]:
'''
#create a cloud free composite with defualt parameters
composite = ee.Algorithms.Landsat.simpleComposite(landsat_8)

#create a cloud free composite with custom parameters
customComposite = ee.Algorithms.Landsat.simpleComposite({landsat_8: landsat_8,percentile: 75,cloudScoreRange: 5
})

#display the composites
Map.addLayer(composite, {bands: ['B4', 'B3', 'B2'], max: 128}, 'TOA composite')
Map.addLayer(customComposite, {bands: ['B4', 'B3', 'B2'], max: 128},'Custom TOA composite')
Map.setCenter(28.75, -0.66667, 8) #centering map on North Kivu Region, DRC
Map
''' 

"\n#create a cloud free composite with defualt parameters\ncomposite = ee.Algorithms.Landsat.simpleComposite(landsat_8)\n\n#create a cloud free composite with custom parameters\ncustomComposite = ee.Algorithms.Landsat.simpleComposite({landsat_8: landsat_8,percentile: 75,cloudScoreRange: 5\n})\n\n#display the composites\nMap.addLayer(composite, {bands: ['B4', 'B3', 'B2'], max: 128}, 'TOA composite')\nMap.addLayer(customComposite, {bands: ['B4', 'B3', 'B2'], max: 128},'Custom TOA composite')\nMap.setCenter(28.75, -0.66667, 8) #centering map on North Kivu Region, DRC\nMap\n"

### Forest Change Analysis

In [10]:
#importing the Hansen Global Forest Change Dataset
gfc2013 = ee.Image('UMD/hansen/global_forest_change_2014')
Map.addLayer(gfc2013, {}, "UMD/hansen/global_forest_change_2014")
Map

Map(bottom=33189.0, center=[-0.66667, 28.75], controls=(WidgetControl(options=['position', 'transparent_bg'], …

The reason the image looks red is that the first three bands are treecover2000, loss, and gain. The treecover2000 band is expressed as a percent and has values much higher than loss (green) and gain (blue) which are binary ({0, 1}). The image therefore displays as overwhelmingly red.

You can find the full band list at: https://developers.google.com/earth-engine/tutorials/tutorial_forest_02

In [11]:
#putting everything together
treeCover = gfc2013.select(['treecover2000'])
lossImage = gfc2013.select(['loss'])
gainImage = gfc2013.select(['gain'])

#Add the tree cover layer in green.
Map.addLayer(treeCover.mask(treeCover),
    {'palette': ['000000', '00FF00'], 'max': 100}, 'Forest Cover')

#Add the loss layer in red.
Map.addLayer(lossImage.mask(lossImage),
            {'palette': ['FF0000']}, 'Loss')

#Add the gain layer in blue.
Map.addLayer(gainImage.mask(gainImage),
            {'palette': ['0000FF']}, 'Gain')

Map.setCenter(28.75, -0.66667, 8) #centering map on North Kivu Region, DRC


Map

Map(bottom=33189.0, center=[-0.66667, 28.75], controls=(WidgetControl(options=['position', 'transparent_bg'], …

### Quantifying Forest Change

The Hansen et al. dataset has a band whose pixels are 1 where loss occurred and 0 otherwise (loss) and a band that is 1 where gain has occurred and a 0 otherwise (gain).

The key method for calculating statistics in an image region in GEE is reduceRegion(). For example, suppose we want to calculate the number of pixels estimated to represent forest loss during the study period. 

In [14]:
#Sum the values of forest loss pixels in the Congo Republic.
forest_loss_stats = lossImage.reduceRegion({
  'reducer': ee.Reducer.sum(),
  'geometry': north_kivu_DRC,
  'scale': 30
});
print(stats);

EEException: Invalid argument for ee.Reducer(): ({'reducer': <ee.Reducer object at 0x7fcc9b11cb00>, 'geometry': GeoJSON(data={'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[29.970703124999996, 0.7415564444349354], [29.904785156249996, 0.8129610018708315], [29.8553466796875, 0.8074683870228081], [29.827880859375, 0.8733792609738106], [29.838867187500004, 0.9337965488500297], [29.783935546874996, 0.9008417889908868], [29.7235107421875, 0.8898568022677679], [29.696044921875, 0.8514090937773031], [29.6246337890625, 0.884364296613886], [29.619140624999996, 0.856901647439813], [29.608154296874996, 0.8349313860427184], [29.520263671875, 0.8459165322899798], [29.5037841796875, 0.8129610018708315], [29.50927734375, 0.7690198781221326], [29.4708251953125, 0.7909904981540058], [29.448852539062504, 0.8294388013837057], [29.3829345703125, 0.7745125438579877], [29.344482421875, 0.7086000996551165], [29.454345703125, 0